**0. Trazendo os arquivos do Drive **

In [ ]:
#criar uma pasta com os arquivos do drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%bash
#copiando e colando os arquivos para análise do Google Drive para esse ambiente de trabalho.
cp /content/drive/MyDrive/Atividade_Complementar/Chr7-reference.fasta.gz .
cp /content/drive/MyDrive/Atividade_Complementar/caso3-tumor-pulmonar_R1.fastq.gz .
cp /content/drive/MyDrive/Atividade_Complementar/caso3-tumor-pulmonar_R2.fastq.gz .

**0.5 Configuração de Ambiente Padrão**

In [ ]:
%%bash
# Instalar miniconda e mamba
wget --quiet https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
bash Miniforge3-Linux-x86_64.sh -b -p /usr/local -u
mamba init
mamba config set auto_activate_base true
mamba shell init --shell bash --root-prefix=~/.local/share/mamba
rm Miniforge3-Linux-x86_64.sh

In [ ]:
%%bash
# Instalar os software
mamba install -c bioconda -c conda-forge fastqc bwa gatk4 cutadapt samtools bedtools ensembl-vep=115 -y

In [ ]:
%%bash
# Criar as pastas para organizar os arquivos que serão gerados
mkdir -p raw fastqc cutadapt bwa gatk mutect vep reference

In [ ]:
%%bash
# Mover arquivos para as suas respectivas pastas e descompactar fasta.gz
mv *.fastq.gz raw/
gunzip -c Chr7-reference.fasta.gz > reference/Chr7-reference.fasta
rm -rf Chr7-reference.fasta.gz sample_data/

**1. Controle de Qualidade**

In [ ]:
%%bash
# Gerar o relatório da qualidade dos reads de sequenciamento
fastqc -o fastqc/ raw/caso3-tumor-pulmonar_R1.fastq.gz raw/caso3-tumor-pulmonar_R2.fastq.gz

In [ ]:
%%bash
# Filtrar sequências muito pequenas e cortar as pontas
cutadapt \
  -u 5 -U 5 \
  -u -5 -U -5 \
  -m 90 \
  -o cutadapt/caso3-tumor-pulmonar-trimmed_R1.fastq.gz \
  -p cutadapt/caso3-tumor-pulmonar-trimmed_R2.fastq.gz \
  raw/caso3-tumor-pulmonar_R1.fastq.gz \
  raw/caso3-tumor-pulmonar_R2.fastq.gz

**2. Mapeamento com a referência do genoma humano**

In [ ]:
%%bash
# Indexar o genoma humano de referência para realizar o mapeamento e posterior remoção de contaminante (cerca de 5 min)
samtools faidx reference/Chr7-reference.fasta
bwa index reference/Chr7-reference.fasta

In [ ]:
%%bash
# Proceder com o alinhamento contra o genoma humano
bwa mem reference/Chr7-reference.fasta \
  -R '@RG\tID:sample1\tSM:sample1\tPL:ILLUMINA\tLB:lib1\tPU:unit1' \
  cutadapt/caso3-tumor-pulmonar-trimmed_R1.fastq.gz \
  cutadapt/caso3-tumor-pulmonar-trimmed_R2.fastq.gz \
  | samtools sort -o bwa/caso3-tumor-pulmonar-mapped.bam

In [ ]:
%%bash
# Indexar o bam para gerar o bai
samtools index bwa/caso3-tumor-pulmonar-mapped.bam

**3. Chamada de Variantes **

In [ ]:
%%bash
# Indexar a referência e BAM para chamada de variantes com GATK
samtools dict reference/Chr7-reference.fasta > reference/Chr7-reference.dict
samtools index bwa/caso3-tumor-pulmonar-mapped.bam

In [ ]:
%%bash
# Chamada de variantes com mutect2 --> somática
 gatk Mutect2 -R reference/Chr7-reference.fasta \
    -I bwa/caso3-tumor-pulmonar-mapped.bam \
    -O mutect/caso3-tumor-pulmonar-unfiltered.vcf

In [ ]:
%%bash
# Filtragem de variantes
gatk FilterMutectCalls \
    -R reference/Chr7-reference.fasta \
    -V mutect/caso3-tumor-pulmonar-unfiltered.vcf \
    -O mutect/caso3-tumor-pulmonar-filtered.vcf

**4. Anotação de Variantes **

In [ ]:
%%bash
# Baixa os arquivos do genoma humano (Homo sapiens, versão GRCh38)
# Esses arquivos são usados pelo VEP para entender e anotar as mutações encontradas
# A opção --NO_UPDATE serve para não atualizar nada pela internet e deixar o processo mais rápido
vep_install -a cf -s homo_sapiens -y GRCh38 --NO_UPDATE

In [ ]:
%%bash
# Index VCF files para anotação
bgzip mutect/caso3-tumor-pulmonar-filtered.vcf
tabix -f -p vcf mutect/caso3-tumor-pulmonar-filtered.vcf.gz


In [ ]:
%%bash
# Anotação de variantes a nível de proteína
vep -i mutect/caso3-tumor-pulmonar-filtered.vcf.gz \
    --output_file vep/caso3-tumor-pulmonar-filtered.vep.tsv \
    --tab --force_overwrite \
    --offline --cache --dir_cache ~/.vep \
    --assembly GRCh38 \
    --symbol --hgvs --canonical --af_1kg --max_af \
    --everything --pick\
    --stats_file vep/caso3-tumor-pulmonar-filtered.vep.html


**5. Acionamento para Tratamento **

2 variantes foram encontradas no paciente
1) L858R no gene EGFR
2) V600E no gene BRAF


O gene EGFR é um gene que codifica uma proteína receptora de membrana chamada Receptor do Fator de Crescimento Epidérmico

- A mutação apresentada: substituição de leucina (L) por arginina (R) no códon 858, exon 21/28 --> Isso gera um ganho de função na tirosina quinase do EGFR causando um aumento na sinalização celular contínua, gerando maior proliferação e sobrevivência eclular

- Essa alteração é altamente associada a adenocarcinoma de pulmão (NSCLC)

- Para tratamento dele é interessante a utilização de inibidores de tirosina quinase, mas atualmente existe uma nova terapia Datopotamab Deruxtecan, que é um anticorpo monoclonal altamente específico com uma droga citotóxica potente, permitindo entregar o quimioterápico diretamente dentro da célula tumoral, poupando o tecido saudável

O gene BRAF é um gene fundamental em uma das principais vias de sinalização celular do corpo humano, chamada via MAPK/ERK — que controla o crescimento, a divisão e a sobrevivência das células

- A mutação apresentada: substituição de valina (V) por glutamato (E) no códon 600, exon 15/18 --> Isso gera uma ativação constitutiva da via MAPK, levando à proliferação celular incontrolada.

- Essa alteração é altamente associada amelanoma, câncer de tireoide, colorretal e pulmonar

- Para tratamento dele é interessante utilizar medicamentos combinados como Dabrafenibe + Trametinibe ou Encorafenibe + Binimetinibe